# CMS GIWAXS raw data processing & exporting notebook - time resolved GIWAXS series measurements
In this notebook you output xr.DataSets stored as .zarr stores containing all your raw,
remeshed (reciprocal space), and caked CMS GIWAXS data. Saving as a zarr automatically converts the array to a dask array!

In [ ]:
# # Outdated, this used to work to just overwrite existing PyHyper install in JupyterHub conda environment
# # If you need a custom PyHyper version install, you may need your own conda environment

# # Kernel updates if needed, remember to restart kernel after running this cell!:
# !pip install -e /nsls2/users/alevin/repos/PyHyperScattering  # to use pip to install via directory

# Imports

In [ ]:
### Imports:
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import xarray as xr
import PyHyperScattering as phs
import pygix
import gc
from tqdm.auto import tqdm  # progress bar loader!

print(f'Using PyHyperScattering Version: {phs.__version__}')

# Set colormap
cmap = plt.cm.turbo.copy()
cmap.set_bad('black')

# Defining some objects

## Define & check paths

In [ ]:
# I like pathlib for its readability & checkability, it's also necessary for the loadSeries function later on
# Replace the paths with the ones relevant to your data, you can use the ".exists()" method to make sure you defined a path correctly
userPath = pathlib.Path('/nsls2/users/alevin')  # Your users path is great for small items that are personal to you (100 GB limit)
propPath = pathlib.Path('/nsls2/data/cms/proposals/2023-2/pass-311415')  # The proposals path is a good place to store large data (>1 TB space?)
dataPath = propPath.joinpath('KWhite5')
maskponiPath = userPath.joinpath('giwaxs_suite/beamline_data/maskponi')
outPath = propPath.joinpath('AL_processed_data')

# Select poni & mask filepaths
poniFile = maskponiPath.joinpath('LaB6_fixed_rot_x517.2.poni')
maskFile = maskponiPath.joinpath('LaB6.json')

In [ ]:
# List the files inside the dataPath folder
sorted([f.name for f in dataPath.iterdir()])  # list all filenames inside a path

In [ ]:
# Select a time series sample folder from above list

sample = 'pybtz_CBCNp5_15_200_40_60_60_014'
samplePath = dataPath.joinpath(sample, 'maxs/raw')
# sorted([f.name for f in samplePath.iterdir()])  # list all filenames inside a path

## Define sets/lists of filtered filepaths

In [ ]:
# Generate sets for samples with multiple scan ids per series scan
# Some of my series are broken into different scan ids because I changed the exposure time

# Choose series scan id(s)
series_ids = ['1118329', '1118330', '1118331']

# Create separate sets for single vs series measurements, customize per your data:
# I had 3 different scan ids in one series measurement, so I combine them all first 
# before substracting them from the total file list
exp0p1_set = set(samplePath.glob(f'*{series_ids[0]}*')) 
exp0p5_set = set(samplePath.glob(f'*{series_ids[1]}*'))
exp2p0_set = set(samplePath.glob(f'*{series_ids[2]}*'))
qperp_set = set(samplePath.glob('*qperp*'))

series_set = exp0p1_set.union(exp0p5_set, exp2p0_set)
singles_set = set(samplePath.iterdir()).difference(series_set)
qpara_set = singles_set.difference(qperp_set)

# # Check content of sets
# print('qperp images:')
# display(sorted([f.name for f in qperp_set]))

# print('\nqpara images:')
# display(sorted([f.name for f in qpara_set]))

# print('\nimage series:')
# display(sorted([f.name for f in series_set]))

## Define metadata naming schemes & initialize loaders

In [17]:
# My example metadata filename naming schemes:
# Make sure the length of this list lines up with your filenames split by underscore (or however you split them)!

# Metadata naming schemes for the pybtz samples
# For nonrotated, qpara images:
qpara_md_naming_scheme = ['material', 'solvent', 'concentration', 'gap_height', 'blade_speed',
                          'solution_temperature', 'stage_temperature', 'sample_number', 'time_start',
                          'x_position_offset', 'incident_angle', 'exposure_time', 'scan_id', 'detector']

# For rotated, qperp images:
qperp_md_naming_scheme = ['material', 'solvent', 'concentration', 'gap_height', 'blade_speed',
                          'solution_temperature', 'stage_temperature', 'sample_number', 'in-plane_orientation',
                          'time_start', 'x_position_offset', 'incident_angle', 'exposure_time', 'scan_id', 'detector']

# For in situ series images:
series_md_naming_scheme = ['material', 'solvent', 'concentration', 'gap_height', 'blade_speed',
                           'solution_temperature', 'stage_temperature', 'sample_number', 'time_start',
                           'x_position_offset', 'incident_angle', 'exposure_time', 'scan_id', 
                           'series_number', 'detector']

# A way to check our naming schemes to make sure they're right:
delim = '_'
file_sets =    [             qpara_set,              qperp_set,              series_set]
file_schemes = [qpara_md_naming_scheme, qperp_md_naming_scheme, series_md_naming_scheme]

for file_set, file_scheme in zip(file_sets, file_schemes):
    first_filename = sorted(file_set)[0].name
    print(f'\nFilename: {first_filename}')
    first_filename_list = first_filename.split(delim)
    for tup in zip(file_scheme, first_filename_list):
        print(tup)


Filename: pybtz_CBCNp5_15_200_40_60_60_014_1005.1s_x3.002_th0.100_5.00s_1118375_maxs.tiff
('material', 'pybtz')
('solvent', 'CBCNp5')
('concentration', '15')
('gap_height', '200')
('blade_speed', '40')
('solution_temperature', '60')
('stage_temperature', '60')
('sample_number', '014')
('time_start', '1005.1s')
('x_position_offset', 'x3.002')
('incident_angle', 'th0.100')
('exposure_time', '5.00s')
('scan_id', '1118375')
('detector', 'maxs.tiff')

Filename: pybtz_CBCNp5_15_200_40_60_60_014_qperp_1586.6s_x-2.001_th0.100_5.00s_1118405_maxs.tiff
('material', 'pybtz')
('solvent', 'CBCNp5')
('concentration', '15')
('gap_height', '200')
('blade_speed', '40')
('solution_temperature', '60')
('stage_temperature', '60')
('sample_number', '014')
('in-plane_orientation', 'qperp')
('time_start', '1586.6s')
('x_position_offset', 'x-2.001')
('incident_angle', 'th0.100')
('exposure_time', '5.00s')
('scan_id', '1118405')
('detector', 'maxs.tiff')

Filename: pybtz_CBCNp5_15_200_40_60_60_014_544.2s_x0.00

In [18]:
# Initalize CMSGIWAXSLoader objects with the above naming schemes
qpara_loader = phs.load.CMSGIWAXSLoader(md_naming_scheme=qpara_md_naming_scheme)
qperp_loader = phs.load.CMSGIWAXSLoader(md_naming_scheme=qperp_md_naming_scheme)
series_loader = phs.load.CMSGIWAXSLoader(md_naming_scheme=series_md_naming_scheme)

# Data processing

## Single image scans outside of series measurement
Using same single_images_to_dataset function as in the single image processing example notebook
Break up sets below according to your data

### qperp set:

#### intialize integrators

In [ ]:
qperp_recip_integrator = phs.integrate.PGGeneralIntegrator(geomethod = 'ponifile',
                                                           ponifile = poniFile,
                                                           output_space = 'recip',
                                                           inplane_config = 'q_perp')
qperp_caked_integrator = phs.integrate.PGGeneralIntegrator(geomethod = 'ponifile',
                                                           ponifile = poniFile,
                                                           output_space = 'caked',
                                                           inplane_config = 'q_perp')

#### generate, check, save: recip Dataset

In [19]:
# Use the single_images_to_dataset utility function to pygix transform all raw files in an indexable list
# Located in the IntegrationUtils script, CMSGIWAXS class:

# Initalize CMSGIWAXS util object
util = phs.util.IntegrationUtils.CMSGIWAXS(sorted(qperp_set), qperp_loader, qperp_recip_integrator)
raw_DS, recip_DS = util.single_images_to_dataset()  # run function 
display(recip_DS)

Transforming Raw Data:   0%|          | 0/29 [00:00<?, ?it/s]

<xarray.Dataset>
Dimensions:  (q_z: 1043, q_perp: 981)
Coordinates:
  * q_z      (q_z) float64 -1.889 -1.885 -1.88 -1.875 ... 2.963 2.968 2.972
  * q_perp   (q_perp) float64 -2.48 -2.475 -2.47 -2.466 ... 2.235 2.24 2.245
Data variables: (12/30)
    1118405  (q_z, q_perp) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118406  (q_z, q_perp) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118407  (q_z, q_perp) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118408  (q_z, q_perp) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118409  (q_z, q_perp) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118410  (q_z, q_perp) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    ...       ...
    1118429  (q_z, q_perp) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118430  (q_z, q_perp) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118431  (q_z, q_perp) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118432  (q_z, q_perp) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118433  (q_z, q_perp) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118434  (q_z, q_perp) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [ ]:
# # Example of a quick plot check if desired here:
# for DA in tqdm(recip_DS.data_vars.values()):  
#     cmin = 1
#     cmax = DA.quantile(0.999)
    
#     ax = DA.sel(q_perp=slice(-1.1, 2.1), q_z=slice(-0.05, 2.4)).plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), figsize=(8,4))
#     ax.axes.set(aspect='equal', title=f'{DA.material}, incident angle: {DA.incident_angle}, scan id: {DA.scan_id}')
#     plt.show()
#     plt.close('all')

In [ ]:
# # Saving dataset with xarray's to_zarr() method:
# # General structure below:

# # Set where to save file and what to name it
# savePath = outPath.joinpath('testing_zarrs')
# savePath.mkdir(exist_ok=True)
# savename = 'custom_save_name.zarr'

# # Save it
# recip_DS.to_zarr(savePath.joinpath(savename))

#### generate, check, save: caked Dataset

In [20]:
# Use the single_images_to_dataset function to pygix transform all raw files in an indexable list

# Run function, generate caked reciprocal space output
raw_DS, caked_DS = phs.PGGeneralIntegrator.single_images_to_dataset(sorted(qperp_set), qperp_loader, qperp_caked_integrator)
display(caked_DS)

Transforming Raw Data:   0%|          | 0/29 [00:00<?, ?it/s]

<xarray.Dataset>
Dimensions:  (chi: 180, qr: 1000)
Coordinates:
  * chi      (chi) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * qr       (qr) float64 0.0001473 0.003876 0.007604 ... 3.717 3.721 3.725
Data variables: (12/30)
    1118405  (chi, qr) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118406  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118407  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118408  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118409  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118410  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    ...       ...
    1118429  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118430  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118431  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118432  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118433  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1118434  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [ ]:
# # Example of a quick plot check if desired here:
# for DA in tqdm(caked_DS.data_vars.values()):   
#     cmin = DA.quantile(0.01)
#     cmax = DA.quantile(0.99)
    
#     ax = DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), figsize=(8,4))
#     ax.axes.set(title=f'{DA.material}, incident angle: {DA.incident_angle}, scan id: {DA.scan_id}')
#     plt.show()
#     plt.close('all')

In [ ]:
# # Saving dataset with xarray's to_zarr() method:
# # General structure below:

# # Set where to save file and what to name it
# savePath = outPath.joinpath('testing_zarrs')
# savePath.mkdir(exist_ok=True)
# savename = 'custom_save_name.zarr'

# # Save it
# caked_DS.to_zarr(savePath.joinpath(savename))

### qpara set:

#### intialize integrators

In [ ]:
template_xr = qpara_loader.loadSingleImage(sorted(qpara_set)[-1])
qpara_recip_integrator = phs.integrate.PGGeneralIntegrator(geomethod = 'ponifile',
                                                           ponifile = poniFile,
                                                           output_space = 'recip',
                                                           template_xr = template_xr,
                                                           inplane_config = 'q_para')
qpara_caked_integrator = phs.integrate.PGGeneralIntegrator(geomethod = 'ponifile',
                                                           ponifile = poniFile,
                                                           output_space = 'caked',
                                                           template_xr = template_xr,
                                                           inplane_config = 'q_para')

#### generate, check, save: recip Dataset

In [ ]:
# Use the single_images_to_dataset utility function to pygix transform all raw files in an indexable list
# Located in the IntegrationUtils script, CMSGIWAXS class:

# Initalize CMSGIWAXS util object
util = phs.util.IntegrationUtils.CMSGIWAXS(sorted(qpara_set), qpara_loader, qpara_recip_integrator)
raw_DS, recip_DS = util.single_images_to_dataset()  # run function 
display(recip_DS)

In [ ]:
# # Example of a quick plot check if desired here:
# for DA in tqdm(list(recip_DS.data_vars.values())[::8]):   
#     ax = DA.sel(q_para=slice(-1.1, 2.1), q_z=slice(-0.05, 2.4)).plot.imshow(cmap=cmap, norm=plt.Normalize(50, 1000), figsize=(8,4))
#     ax.axes.set(aspect='equal', title=f'{DA.material}, incident angle: {DA.incident_angle}, scan id: {DA.scan_id}')
#     plt.show()
#     plt.close('all')

In [ ]:
# # Saving dataset with xarray's to_zarr() method:
# # General structure below:

# # Set where to save file and what to name it
# savePath = outPath.joinpath('testing_zarrs')
# savePath.mkdir(exist_ok=True)
# savename = 'custom_save_name.zarr'

# # Save it
# recip_DS.to_zarr(savePath.joinpath(savename))

#### generate, check, save: caked Dataset

In [ ]:
# Use the single_images_to_dataset utility function to pygix transform all raw files in an indexable list
# Located in the IntegrationUtils script, CMSGIWAXS class:

# Initalize CMSGIWAXS util object
util = phs.util.IntegrationUtils.CMSGIWAXS(sorted(qpara_set), qpara_loader, qpara_caked_integrator)
raw_DS, caked_DS = util.single_images_to_dataset()  # run function 
display(caked_DS)

In [ ]:
# # Example of a quick plot check if desired here:
# for DA in tqdm(caked_DS.data_vars.values()):   
#     cmin = DA.quantile(0.01)
#     cmax = DA.quantile(0.99)
    
#     ax = DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), figsize=(8,4))
#     ax.axes.set(title=f'{DA.material}, incident angle: {DA.incident_angle}, scan id: {DA.scan_id}')
#     plt.show()
#     plt.close('all')

In [ ]:
# # Saving dataset with xarray's to_zarr() method:
# # General structure below:

# # Set where to save file and what to name it
# savePath = outPath.joinpath('testing_zarrs')
# savePath.mkdir(exist_ok=True)
# savename = 'custom_save_name.zarr'

# # Save it
# caked_DS.to_zarr(savePath.joinpath(savename))

### Series measurement processing

#### Load file series & check raw DataArray

In [21]:
# Check which files you will select by using a given file_filter below
file_filter = '0.10s_1118329'
len([f.name for f in sorted(samplePath.glob(f'*{file_filter}*'))])

100

In [22]:
# Load file series
stacked_dim = 'series_number'  # must be a unique attribute for in each loaded DataArray
loaded_DA = series_loader.loadFileSeries(basepath=samplePath, dims=['series_number'], file_filter=file_filter)  # ensure that your basepath + file_filter selects the filepaths of interest
# NOTE: CMSGIWAXSLoader.loadSeries() is the deprecated method for this. That method accepts either a basepath + filter OR an iterable of filepaths. More details at end of notebook.

# Create coordinates to apply to stacked dimension
time_start = 0
exp_time = np.round(float(loaded_DA.attrs['exposure_time'][:-1]), 1)
times = loaded_DA.coords['series_number'].data.astype('float')*exp_time + exp_time + time_start

# Remove system dimension (leftover from generalized stacking), assign desired coordinate to stacked dimension, and set it as the stacked dimension
raw_DA = loaded_DA.unstack('system')
raw_DA = raw_DA.assign_coords({'time': ('series_number', times)})
raw_DA = raw_DA.swap_dims({'series_number': 'time'})
display(raw_DA)

Found 415 total files.
Found 100 files after applying 'file_filter'.


  0%|          | 0/100 [00:00<?, ?it/s]

Loaded 100/100 files


<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


<xarray.DataArray (pix_y: 1043, pix_x: 981, time: 100)>
array([[[ 0,  0,  0, ...,  2,  3,  0],
        [ 0,  0,  1, ...,  1,  1,  0],
        [ 0,  0,  1, ...,  0,  1,  1],
        ...,
        [ 0,  1,  0, ...,  1,  1,  3],
        [ 0,  1,  0, ...,  1,  1,  1],
        [ 1,  0,  0, ...,  1,  0,  2]],

       [[ 0,  1,  0, ...,  2,  1,  1],
        [ 0,  3,  1, ...,  1,  0,  1],
        [ 0,  0,  0, ...,  3,  1,  0],
        ...,
        [ 0,  0,  0, ...,  2,  3,  0],
        [ 0,  0,  1, ...,  1,  1,  2],
        [ 0,  0,  0, ...,  2,  2,  3]],

       [[ 1,  0,  1, ...,  2,  0,  1],
        [ 0,  1,  0, ...,  2,  1,  3],
        [ 0,  0,  2, ...,  3,  0,  1],
        ...,
...
        ...,
        [-2, -2, -2, ..., -2, -2, -2],
        [-2, -2, -2, ..., -2, -2, -2],
        [-2, -2, -2, ..., -2, -2, -2]],

       [[ 0,  0,  0, ...,  0,  0,  0],
        [ 1,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  1, ...,  0,  2,  0],
        ...,
        [-2, -2, -2, ..., -2, -2, -2],
        [-2, -2, -2, ..., -2, -2, -2],
        [-2, -2, -2, ..., -2, -2, -2]],

       [[ 1,  0,  0, ...,  0,  0,  0],
        [ 1,  0,  0, ...,  0,  0,  1],
        [ 0,  0,  1, ...,  0,  1,  1],
        ...,
        [-2, -2, -2, ..., -2, -2, -2],
        [-2, -2, -2, ..., -2, -2, -2],
        [-2, -2, -2, ..., -2, -2, -2]]], dtype=int32)
Coordinates:
    series_number  (time) object '000000' '000001' ... '000098' '000099'
  * pix_x          (pix_x) int64 0 1 2 3 4 5 6 7 ... 974 975 976 977 978 979 980
  * pix_y          (pix_y) int64 0 1 2 3 4 5 6 ... 1037 1038 1039 1040 1041 1042
  * time           (time) float64 0.1 0.2 0.3 0.4 0.5 ... 9.6 9.7 9.8 9.9 10.0
Attributes: (12/16)
    material:              pybtz
    solvent:               CBCNp5
    concentration:         15
    gap_height:            200
    blade_speed:           40
    solution_temperature:  60
    ...                    ...
    incident_angle:        th0.120
    exposure_time:         0.10s
    scan_id:               1118329
    series_number:         000000
    detector:              maxs.tiff
    dims_unpacked:         ['series_number']

In [ ]:
# Quick facet plot of selected times
cmin=1
cmax=10
times = np.linspace(0.1, 10, 8)

sliced_DA = raw_DA.sel(time=times, method='nearest')
fg = sliced_DA.plot.imshow(figsize=(18, 6), col='time', col_wrap=4, norm=plt.Normalize(cmin, cmax), cmap=cmap, origin='upper')
fg.cbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)
for axes in fg.axs.flatten():
    axes.set(aspect='equal')

plt.show()

#### Transform data to reciprocal space: cartesian (recip)

In [ ]:
# Reinitialize cartesian integrator if necessary
# incident angle should be set here (default is 0.12 if nothing is entered)
qpara_recip_integrator = phs.integrate.PGGeneralIntegrator(geomethod = 'ponifile',
                                                           ponifile = poniFile,
                                                           output_space = 'recip',
                                                           inplane_config = 'q_para',
                                                           incident_angle = 0.12)

In [23]:
# Apply integrator to integrate the raw image stack
recip_DA = qpara_recip_integrator.integrateImageStack(raw_DA)
display(recip_DA)

  0%|          | 0/100 [00:00<?, ?it/s]

<xarray.DataArray (time: 100, q_z: 1043, q_para: 981)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)
Coordinates:
  * q_z      (q_z) float64 -1.889 -1.885 -1.88 -1.875 ... 2.963 2.968 2.972
  * q_para   (q_para) float64 -2.48 -2.475 -2.47 -2.466 ... 2.235 2.24 2.245
  * time     (time) float64 0.1 0.2 0.3 0.4 0.5 0.6 ... 9.5 9.6 9.7 9.8 9.9 10.0
Attributes: (12/16)
    material:              pybtz
    solvent:               CBCNp5
    concentration:         15
    gap_height:            200
    blade_speed:           40
    solution_temperature:  60
    ...                    ...
    incident_angle:        th0.120
    exposure_time:         0.10s
    scan_id:               1118329
    series_number:         000000
    detector:              maxs.tiff
    dims_unpacked:         ['series_number']

In [ ]:
# Facet plot of selected times
cmin = 1
cmax = 10
times = np.linspace(0.1, 10, 8)

sliced_DA = recip_DA.sel(time=times, method='nearest')
fg = sliced_DA.plot.imshow(figsize=(18, 6), col='time', col_wrap=4, norm=plt.Normalize(cmin, cmax), cmap=cmap)
fg.cbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)
for axes in fg.axs.flatten():
    axes.set(aspect='equal')

plt.show()

In [ ]:
# # Saving dataset with xarray's to_zarr() method:
# # General structure below:

# # Set where to save file and what to name it
# savePath = outPath.joinpath('testing_zarrs')
# savePath.mkdir(exist_ok=True)
# savename = 'custom_save_name.zarr'

# # Save it
# recip_DS.to_zarr(savePath.joinpath(savename))

#### Transform data to reciprocal space: polar (caked)

In [ ]:
# Reinitialize cartesian integrator if necessary
qpara_caked_integrator = phs.integrate.PGGeneralIntegrator(geomethod = 'ponifile',
                                                           ponifile = poniFile,
                                                           output_space = 'caked',
                                                           incident_angle = 0.12)

In [24]:
# Apply integrator to integrate the raw image stack
caked_DA = qpara_caked_integrator.integrateImageStack(raw_DA)
caked_DA

  0%|          | 0/100 [00:00<?, ?it/s]

<xarray.DataArray (time: 100, chi: 180, qr: 1000)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)
Coordinates:
  * chi      (chi) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * qr       (qr) float64 0.0001473 0.003876 0.007604 ... 3.717 3.721 3.725
  * time     (time) float64 0.1 0.2 0.3 0.4 0.5 0.6 ... 9.5 9.6 9.7 9.8 9.9 10.0
Attributes: (12/17)
    material:              pybtz
    solvent:               CBCNp5
    concentration:         15
    gap_height:            200
    blade_speed:           40
    solution_temperature:  60
    ...                    ...
    exposure_time:         0.10s
    scan_id:               1118329
    series_number:         000000
    detector:              maxs.tiff
    dims_unpacked:         ['series_number']
    inplane_config:        q_xy

In [ ]:
# Facet plot of selected times
cmin = 1
cmax = 10
times = np.linspace(0.1, 10, 8)

sliced_DA = caked_DA.sel(time=times, method='nearest')
fg = sliced_DA.plot.imshow(figsize=(18, 6), col='time', col_wrap=4, norm=plt.Normalize(cmin, cmax), cmap=cmap)
fg.cbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)
# for axes in fg.axs.flatten():
#     axes.set(aspect='equal')

plt.show()

In [ ]:
# # Saving dataset with xarray's to_zarr() method:
# # General structure below:

# # Set where to save file and what to name it
# savePath = outPath.joinpath('testing_zarrs')
# savePath.mkdir(exist_ok=True)
# savename = 'custom_save_name.zarr'

# # Save it
# recip_DS.to_zarr(savePath.joinpath(savename))

## Not fully implemented or deprecated

### Yoneda peak check 
This can be used as a way to verify / refine your correct beam center y position. The yoneda peak should always appear at a q value corresponding to your incident angle plus your film's critical angle. Check where it is supposed to appera and compare with experimental data, then tweak the beamcenter y position accordingly.

In [ ]:
def yonda_qz(wavelength, alpha_crit, alpha_incidents):
    """Calculate the yoneda qz values given the wavelength, critical angle, and incident angle (in degrees)"""
    qz_inv_meters = ((4 * np.pi) / (wavelength)) * (np.sin(np.deg2rad((alpha_incidents + alpha_crit)/2)))
    qz_inv_angstroms = qz_inv_meters / 1e10
    return qz_inv_angstroms


wavelength = 9.762535309700809e-11  # 12.7 keV
alpha_crit = 0.11  # organic film critical angle
alpha_incidents = np.array([0.08, 0.1, 0.12, 0.15])  # incident angle(s)

yoneda_angles = alpha_incidents + alpha_crit

qz(wavelength, alpha_crit, alpha_incidents)  # expected yoneda qz positions

In [ ]:
# Helper functions:
def select_attrs(data_arrays_iterable, selected_attrs_dict):
    """
    Selects data arrays whose attributes match the specified values.

    Parameters:
    data_arrays_iterable: Iterable of xarray.DataArray objects.
    selected_attrs_dict: Dictionary where keys are attribute names and 
                         values are the attributes' desired values.

    Returns:
    List of xarray.DataArray objects that match the specified attributes.
    """    
    sublist = list(data_arrays_iterable)
    
    for attr_name, attr_values in selected_attrs_dict.items():
        sublist = [da for da in sublist if da.attrs[attr_name] in attr_values]
                
    return sublist

def poni_centers(poniFile, pix_size=0.000172):
    """
    Returns poni center value and the corresponding pixel position. Default pixel size is 172 microns (Pilatus 1M)
    
    This info could be loaded better using default pyFAI methods.
    
    Inputs: poniFile as pathlib path object to the poni file
    Outputs: ((poni1, y_center), (poni2, x_center))
    """
    
    with poniFile.open('r') as f:
        lines = list(f.readlines())
    poni1_str = lines[6]
    poni2_str = lines[7]

    poni1 = float(poni1_str.split(' ')[1])
    poni2 = float(poni2_str.split(' ')[1])

    y_center = poni1 / pix_size
    x_center = poni2 / pix_size
        
    return ((poni1, y_center), (poni2, x_center))

# poni_centers(poniFile)

In [ ]:
# 2D reciprocal space cartesian plots
qxy_min = -1.1
qxy_max = 2.1
qz_min = -0.01
qz_max = 2.2

selected_attrs_dict = {'material': ['PM6'], 'solvent': ['CBCN']}
# selected_attrs_dict = {}

selected_DAs = select_attrs(fixed_recip_DS.data_vars.values(), selected_attrs_dict)
for DA in tqdm(selected_DAs):
    # Slice data for selected q ranges (will need to rename q_xy if dimensions are differently named)
    sliced_DA = DA.sel(q_xy=slice(qxy_min, qxy_max), q_z=slice(qz_min, qz_max))
    
    real_min = float(sliced_DA.compute().quantile(0.05))
    cmin = 1 if real_min < 1 else real_min

    cmax = float(sliced_DA.compute().quantile(0.997))   
    
    # Plot
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), interpolation='antialiased', figsize=(5.5,3.3))
    ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)
    # ax.axes.set(aspect='equal', title=f'Cartesian Plot: {DA.material} {DA.solvent} {DA.rpm}, {float(DA.incident_angle[2:])}° Incidence',
    #             xlabel='q$_{xy}$ [Å$^{-1}$]', ylabel='q$_z$ [Å$^{-1}$]')
    ax.axes.set(aspect='equal', title=f'Cartesian Plot: {DA.material} {DA.solvent}, {float(DA.incident_angle[2:])}° Incidence',
                xlabel='q$_{xy}$ [Å$^{-1}$]', ylabel='q$_z$ [Å$^{-1}$]')
    ax.figure.set(tight_layout=True, dpi=130)
    
    # ax.figure.savefig(savePath.joinpath(f'{DA.material}-{DA.solvent}-{DA.rpm}_qxy{qxy_min}to{qxy_max}_qz{qz_min}to{qz_max}_{DA.incident_angle}.png'), dpi=150)
    # ax.figure.savefig(savePath.joinpath(f'{DA.material}-{DA.solvent}_qxy{qxy_min}to{qxy_max}_qz{qz_min}to{qz_max}_{DA.incident_angle}.png'), dpi=150)

    plt.show()
    plt.close('all')

In [ ]:
# Yoneda peak linecut check
qxy_min = 0.22
qxy_max = 2
qz_min = -0.02
qz_max = 0.06

selected_DAs = select_attrs(fixed_recip_DS.data_vars.values(), selected_attrs_dict)
for DA in tqdm(selected_DAs):
    # Slice data for selected q ranges (will need to rename q_xy if dimensions are differently named)
    sliced_DA = DA.sel(q_xy=slice(qxy_min, qxy_max), q_z=slice(qz_min, qz_max))
    qz_integrated_DA = sliced_DA.sum('q_xy')
    
    # Plot
    qz_integrated_DA.plot.line(label=DA.incident_angle)
    
plt.legend()
plt.grid(visible=True, which='major', axis='x')
plt.show()

### phs.CMSGIWAXS.loadSeries()
If it's easier to sort/filter/organize files outside of the loadFileSeries filter arguments. The option to enter a list of filepaths into loadFileSeries will be implemented soon.

If integrating time series data, the legacy loadSeries method will accept these iterables for loading the data stack. 

In [ ]:
raw_DA = series_loader.loadSeries(sorted(exp0p1_set))
display(raw_DA)